In [28]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Libraries loaded!")

✅ Libraries loaded!


In [29]:
df = pd.read_csv('/Users/yashakhorsandi/Desktop/پروژه دیوار/real_estate_ads.csv')

/var/folders/j2/6c3r7nc116sgr13bnd1dxtth0000gn/T/ipykernel_42299/796676103.py:1: DtypeWarning: Columns (10,26,28,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/yashakhorsandi/Desktop/پروژه دیوار/real_estate_ads.csv')


In [57]:
# Drop requested columns and update df
cols_to_drop = [
    'location_radius', 'regular_person_capacity', 'extra_person_capacity',
    'cost_per_extra_person', 'rent_price_on_regular_days',
    'rent_price_on_special_days', 'rent_price_at_weekends', 'has_water', 'has_warm_water_provider', 'has_electricity', 'has_gas',
    'has_heating_system', 'has_cooling_system', 'has_restroom',
    'has_security_guard', 'has_barbecue', 'building_direction', 'has_pool',
    'has_jacuzzi', 'has_sauna', 'floor_material', 'property_type'
]

present = [c for c in cols_to_drop if c in df.columns]
if present:
    df = df.drop(columns=present)
    print(f"Dropped {len(present)} columns: {present}")
else:
    print("No target columns found to drop.")

print(f"After drops — df shape: {df.shape}")

# --- Now filter dataset for city + categories ---
print('\nFiltering dataset for: city_slug==tehran, cat2_slug==residential-rent, cat3_slug==apartment-rent')
mask = pd.Series(True, index=df.index)
cols_checked = []

if 'city_slug' in df.columns:
    mask &= (df['city_slug'] == 'tehran')
    cols_checked.append('city_slug')
else:
    print("Warning: 'city_slug' column not found — skipping that filter")

if 'cat2_slug' in df.columns:
    mask &= (df['cat2_slug'] == 'residential-rent')
    cols_checked.append('cat2_slug')
else:
    print("Warning: 'cat2_slug' column not found — skipping that filter")

if 'cat3_slug' in df.columns:
    mask &= (df['cat3_slug'] == 'apartment-rent')
    cols_checked.append('cat3_slug')
else:
    print("Warning: 'cat3_slug' column not found — skipping that filter")

if not cols_checked:
    print("No filter columns available — df unchanged")
else:
    before = df.shape[0]
    df = df[mask].copy().reset_index(drop=True)
    after = df.shape[0]
    print(f"Applied filters on: {cols_checked}")
    print(f"Rows before: {before}, after: {after}, removed: {before - after}")
    print(f"Updated df shape: {df.shape}")

# --- Drop additional transformable / deed fields requested by user ---
extra_drop = [
    'transformable_price','transformable_credit','transformed_credit',
    'transformable_rent','transformed_rent','land_size','deed_type','has_business_deed'
]
present_extra = [c for c in extra_drop if c in df.columns]
if present_extra:
    before2 = df.shape[0]
    df = df.drop(columns=present_extra)
    after2 = df.shape[0]
    print(f"Dropped {len(present_extra)} additional columns: {present_extra}")
    print(f"Rows unchanged (only columns dropped). df shape now: {df.shape}")
else:
    print("No additional transformable/deed columns found to drop.")

# Final status
print(f"Final df shape: {df.shape}")

Dropped 22 columns: ['location_radius', 'regular_person_capacity', 'extra_person_capacity', 'cost_per_extra_person', 'rent_price_on_regular_days', 'rent_price_on_special_days', 'rent_price_at_weekends', 'has_water', 'has_warm_water_provider', 'has_electricity', 'has_gas', 'has_heating_system', 'has_cooling_system', 'has_restroom', 'has_security_guard', 'has_barbecue', 'building_direction', 'has_pool', 'has_jacuzzi', 'has_sauna', 'floor_material', 'property_type']
After drops — df shape: (1000000, 38)

Filtering dataset for: city_slug==tehran, cat2_slug==residential-rent, cat3_slug==apartment-rent
Applied filters on: ['city_slug', 'cat2_slug', 'cat3_slug']
Rows before: 1000000, after: 64630, removed: 935370
Updated df shape: (64630, 38)
Dropped 8 additional columns: ['transformable_price', 'transformable_credit', 'transformed_credit', 'transformable_rent', 'transformed_rent', 'land_size', 'deed_type', 'has_business_deed']
Rows unchanged (only columns dropped). df shape now: (64630, 30)


In [58]:
# Drop additional rent-related columns and update df
drop_cols_2 = ['rent_mode', 'rent_to_single', 'rent_type', 'price_mode', 'credit_mode']
present2 = [c for c in drop_cols_2 if c in df.columns]
if present2:
    df = df.drop(columns=present2)
    print(f"Dropped {len(present2)} columns: {present2}")
else:
    print("No additional rent-related columns found to drop.")

print(f"Updated df shape after additional drops: {df.shape}")

Dropped 5 columns: ['rent_mode', 'rent_to_single', 'rent_type', 'price_mode', 'credit_mode']
Updated df shape after additional drops: (64630, 25)


In [31]:
df.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,user_type,description,title,rent_mode,rent_value,...,property_type,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,مشاور املاک,۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم ...,باغ ویلا اجاره روزانه استخر داخل لشکرآباد سهیلیه,NaN,NaN,...,NaN,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0
1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,مشاور املاک,دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nب...,۶۰ متر قلهک فول امکانات,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0
2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,NaN,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,مقطوع,26000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN
3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,NaN,فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق...,فرشته ۹۰ متر دفتر کار مدرن موقعیت اداری,مقطوع,95000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,مشاور املاک,هلدینگ ساختمانی اکبری\n\nهمراه شما هستیم برای ...,۱۱۵ متری/شمالی رو به آفتاب/اکبری,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df.head(50)

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,user_type,description,title,rent_mode,rent_value,...,total_floors_count,unit_per_floor,has_balcony,has_elevator,has_warehouse,has_parking,construction_year,is_rebuilt,location_latitude,location_longitude
0,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,NaN,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,مقطوع,26000000.0,...,NaN,NaN,NaN,True,True,True,۱۴۰۱,False,35.703865,51.373459
1,residential-rent,apartment-rent,tehran,gisha,2024-08-01 00:00:00,مشاور املاک,‼️‼️ارزنده ترین واحد منطقه ‼️‼️ \n\n✅ 91 متر ب...,گیشا / ۹۱ متر /فرعی زوج مشرف به پارک گفتگو/ ار...,مجانی,0.0,...,5,NaN,true,False,True,True,۱۳۸۵,True,35.733952,51.380608
2,residential-rent,apartment-rent,tehran,majidiyeh,2024-11-01 00:00:00,مشاور املاک,60 متر آپارتمان \n\nیک خوابه \n\nطبقه دوم \n\n...,60 متر یک خوابه,مقطوع,15000000.0,...,NaN,NaN,NaN,False,False,True,۱۳۸۳,True,35.735909,51.464397
3,residential-rent,apartment-rent,tehran,west-tehranpars,2024-07-01 00:00:00,مشاور املاک,✅.....املاک بزرگ تهرانپارس:سام.....✅\n\n\n86 م...,86متر*2خواب*کلید نخورده*فول*بهترین لوکیشن,مجانی,0.0,...,5,3,true,True,True,True,۱۴۰۰,True,35.739414,51.525150
4,residential-rent,apartment-rent,tehran,tehran-kerman,2024-06-01 00:00:00,NaN,این واحد در طبقه همکف یک ساختمان دوطبقه واقع ش...,اپارتمان ۸۰ متری بازسازی کامل یکخواب,مقطوع,10000000.0,...,2,1,NaN,False,True,False,۱۳۷۵,True,35.729465,51.468460
5,residential-rent,apartment-rent,tehran,abouzar,2024-06-01 00:00:00,مشاور املاک,✅پارکینگ\n✅انباری\n✅بالکن\n✅آسانسور\n✅دو خوابه...,آپارتمان ۸۵ متری نبرد جنوبی نصر,مقطوع,15000000.0,...,5,4,true,True,True,True,۱۳۹۶,NaN,35.657238,51.472324
6,residential-rent,apartment-rent,tehran,jordan,2024-12-01 00:00:00,NaN,✔️به نام یگانه معمار هستی\n✔️ فتحی هستم کارگزا...,۱۱۵ متر مشابه نوساز جردن ولیعصر,مجانی,0.0,...,NaN,NaN,NaN,True,True,True,۱۴۰۰,False,35.768757,51.419998
7,residential-rent,apartment-rent,tehran,pirouzi,2024-05-01 00:00:00,مشاور املاک,⬅️چهار طبقه تک واحدی طبقه دوم\n⬅️کابینت MDF\n⬅...,۶۵متر/تک واحد/دریاباری/آسانسور/تخلیه,مقطوع,4000000.0,...,NaN,NaN,NaN,True,True,False,۱۳۹۹,NaN,NaN,NaN
8,residential-rent,apartment-rent,tehran,hekmat,2024-10-01 00:00:00,NaN,♻️♦️کل عکسهای واحد موجود میباشد♻️\n\n✅️ مشخصات...,اندرزگو ۱۶۰ متری,مقطوع,60000000.0,...,6,2,false,True,True,True,۱۴۰۱,False,35.799187,51.445717
9,residential-rent,apartment-rent,tehran,sarsabil,2024-09-01 00:00:00,مشاور املاک,✔️110متر/دو خواب/طبقه سوم\n✔️خوش نقشه و نورگیر...,آپارتمان110متری/دوخوابه/دامپزشکی تقاطع خوش,مجانی,0.0,...,NaN,NaN,true,True,False,True,۱۴۰۳,NaN,35.693245,51.373089


In [39]:
df.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,user_type,description,title,rent_value,price_value,...,total_floors_count,unit_per_floor,has_balcony,has_elevator,has_warehouse,has_parking,construction_year,is_rebuilt,location_latitude,location_longitude
0,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,NaN,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,26000000.0,NaN,...,NaN,NaN,NaN,True,True,True,۱۴۰۱,False,35.703865,51.373459
1,residential-rent,apartment-rent,tehran,gisha,2024-08-01 00:00:00,مشاور املاک,‼️‼️ارزنده ترین واحد منطقه ‼️‼️ \n\n✅ 91 متر ب...,گیشا / ۹۱ متر /فرعی زوج مشرف به پارک گفتگو/ ار...,0.0,NaN,...,5,NaN,true,False,True,True,۱۳۸۵,True,35.733952,51.380608
2,residential-rent,apartment-rent,tehran,majidiyeh,2024-11-01 00:00:00,مشاور املاک,60 متر آپارتمان \n\nیک خوابه \n\nطبقه دوم \n\n...,60 متر یک خوابه,15000000.0,NaN,...,NaN,NaN,NaN,False,False,True,۱۳۸۳,True,35.735909,51.464397
3,residential-rent,apartment-rent,tehran,west-tehranpars,2024-07-01 00:00:00,مشاور املاک,✅.....املاک بزرگ تهرانپارس:سام.....✅\n\n\n86 م...,86متر*2خواب*کلید نخورده*فول*بهترین لوکیشن,0.0,NaN,...,5,3,true,True,True,True,۱۴۰۰,True,35.739414,51.525150
4,residential-rent,apartment-rent,tehran,tehran-kerman,2024-06-01 00:00:00,NaN,این واحد در طبقه همکف یک ساختمان دوطبقه واقع ش...,اپارتمان ۸۰ متری بازسازی کامل یکخواب,10000000.0,NaN,...,2,1,NaN,False,True,False,۱۳۷۵,True,35.729465,51.468460


In [45]:
# Missing values summary (counts + percentages)
print("Computing missing value counts and percentages for current df...")
missing_counts = df.isnull().sum().sort_values(ascending=False)
missing_percent = (missing_counts / len(df) * 100).round(2)
missing_df = pd.concat([missing_counts, missing_percent], axis=1)
missing_df.columns = ['missing_count', 'missing_percent']

# Show only columns with missing values
missing_present = missing_df[missing_df['missing_count'] > 0]
if missing_present.empty:
    print("No missing values found — df is complete.")
else:
    print(f"Columns with missing values: {len(missing_present)}")
    display(missing_present)

# Also save to a CSV for quick review if needed
missing_report_file = '/Users/yashakhorsandi/Desktop/پروژه دیوار/missing_values_report.csv'
missing_present.to_csv(missing_report_file)
print(f"Missing-values report saved to: {missing_report_file}")

Computing missing value counts and percentages for current df...
No missing values found — df is complete.
Missing-values report saved to: /Users/yashakhorsandi/Desktop/پروژه دیوار/missing_values_report.csv


In [59]:
# Drop price_value, drop rows missing rent_value or credit_value, impute lat/lon by neighborhood centroid
print("Starting: drop price_value, drop rows with missing rent/credit, impute lat/lon by neighborhood centroid")

# Drop price_value column if present
if 'price_value' in df.columns:
    df = df.drop(columns=['price_value'])
    print("Dropped column: price_value")
else:
    print("price_value not present — nothing to drop")

# Drop rows missing rent_value or credit_value
before_rows = df.shape[0]
cols_to_require = [c for c in ['rent_value', 'credit_value'] if c in df.columns]
if cols_to_require:
    df = df.dropna(subset=cols_to_require).reset_index(drop=True)
    after_rows = df.shape[0]
    print(f"Dropped {before_rows - after_rows} rows missing {cols_to_require}. New shape: {df.shape}")
else:
    print("rent_value/credit_value columns not present — skipping row drops")

# Ensure latitude/longitude numeric
if 'location_latitude' in df.columns:
    df['location_latitude'] = pd.to_numeric(df['location_latitude'], errors='coerce')
if 'location_longitude' in df.columns:
    df['location_longitude'] = pd.to_numeric(df['location_longitude'], errors='coerce')

# Impute missing lat/lon by neighborhood centroid
if 'neighborhood_slug' in df.columns and 'location_latitude' in df.columns and 'location_longitude' in df.columns:
    # compute centroids from available data
    centroids = df.groupby('neighborhood_slug')[['location_latitude','location_longitude']].mean()
    lat_map = centroids['location_latitude'].to_dict()
    lon_map = centroids['location_longitude'].to_dict()

    df['location_latitude'] = df['location_latitude'].fillna(df['neighborhood_slug'].map(lat_map))
    df['location_longitude'] = df['location_longitude'].fillna(df['neighborhood_slug'].map(lon_map))

    # Fill any remaining missing with global mean
    lat_mean = df['location_latitude'].mean()
    lon_mean = df['location_longitude'].mean()
    df['location_latitude'] = df['location_latitude'].fillna(lat_mean)
    df['location_longitude'] = df['location_longitude'].fillna(lon_mean)

    print("Imputed location_latitude and location_longitude by neighborhood centroids (fallback to global mean)")
    # report remaining missing
    lat_missing = int(df['location_latitude'].isnull().sum())
    lon_missing = int(df['location_longitude'].isnull().sum())
    print(f"Remaining missing - lat: {lat_missing}, lon: {lon_missing}")
else:
    print("Not all required columns for imputation exist (neighborhood_slug and lat/lon) — skipping imputation")

print(f"Final df shape: {df.shape}")

# --- Drop 'user_type' column as requested ---
if 'user_type' in df.columns:
    df = df.drop(columns=['user_type'])
    print("Dropped column: user_type")
else:
    print("Column 'user_type' not found; nothing to drop")

print(f"Final df shape after dropping user_type: {df.shape}")

# --- Drop requested columns: total_floors_count, unit_per_floor, has_balcony ---
drop_cols_3 = ['total_floors_count', 'unit_per_floor', 'has_balcony']
present3 = [c for c in drop_cols_3 if c in df.columns]
if present3:
    df = df.drop(columns=present3)
    print(f"Dropped {len(present3)} columns: {present3}")
else:
    print("No target columns found among total_floors_count/unit_per_floor/has_balcony to drop.")

# Impute missing is_rebuilt with False
if 'is_rebuilt' in df.columns:
    missing_before = df['is_rebuilt'].isnull().sum()
    df['is_rebuilt'] = df['is_rebuilt'].fillna(False)
    missing_after = df['is_rebuilt'].isnull().sum()
    print(f"Filled missing 'is_rebuilt' values: before={missing_before}, after={missing_after}")
else:
    print("Column 'is_rebuilt' not found — skipping imputation")

print(f"Final df shape after these operations: {df.shape}")

# --- Compute property_age from construction_year stored in Farsi digits ---
# target: property_age = 1404 - construction_year
import re

def persian_to_integer(val):
    """Convert values with Persian/Arabic-Indic digits to integer. Return np.nan if cannot parse."""
    if pd.isna(val):
        return np.nan
    s = str(val)
    # mapping for Persian and Arabic-Indic digits
    mapping = {
        '۰': '0','۱': '1','۲': '2','۳': '3','۴': '4','۵': '5','۶': '6','۷': '7','۸': '8','۹': '9',
        '٠': '0','١': '1','٢': '2','٣': '3','٤': '4','٥': '5','٦': '6','٧': '7','٨': '8','٩': '9'
    }
    trans = str.maketrans(mapping)
    s2 = s.translate(trans)
    # remove non-digit characters
    s2 = re.sub(r"[^0-9]", "", s2)
    if s2 == '':
        return np.nan
    try:
        return int(s2)
    except Exception:
        return np.nan

# Apply conversion
if 'construction_year' in df.columns:
    # create a numeric column from possible Persian strings or existing numeric
    df['_construction_year_num'] = df['construction_year'].apply(persian_to_integer)
    # if still NaN, try coercing original directly
    mask_na = df['_construction_year_num'].isna() & df['construction_year'].notna()
    if mask_na.any():
        coerced = pd.to_numeric(df.loc[mask_na, 'construction_year'], errors='coerce')
        df.loc[mask_na, '_construction_year_num'] = coerced

    # compute property_age
    df['property_age'] = 1404 - df['_construction_year_num']
    # invalid ages -> NaN
    df.loc[df['property_age'] < 0, 'property_age'] = np.nan
    df.loc[df['property_age'] > 1404, 'property_age'] = np.nan

    # drop helper column
    df = df.drop(columns=['_construction_year_num'])
    print(f"Computed 'property_age' from 'construction_year'. Examples:\n", df['property_age'].head())
else:
    print("Column 'construction_year' not found — cannot compute property_age")

Starting: drop price_value, drop rows with missing rent/credit, impute lat/lon by neighborhood centroid
Dropped column: price_value
Dropped 432 rows missing ['rent_value', 'credit_value']. New shape: (64198, 24)
Imputed location_latitude and location_longitude by neighborhood centroids (fallback to global mean)
Remaining missing - lat: 0, lon: 0
Final df shape: (64198, 24)
Dropped column: user_type
Final df shape after dropping user_type: (64198, 23)
Dropped 3 columns: ['total_floors_count', 'unit_per_floor', 'has_balcony']
Filled missing 'is_rebuilt' values: before=18584, after=0
Final df shape after these operations: (64198, 20)
Computed 'property_age' from 'construction_year'. Examples:
 0     3.0
1    19.0
2    21.0
3     4.0
4    29.0
Name: property_age, dtype: float64
Computed 'property_age' from 'construction_year'. Examples:
 0     3.0
1    19.0
2    21.0
3     4.0
4    29.0
Name: property_age, dtype: float64


/var/folders/j2/6c3r7nc116sgr13bnd1dxtth0000gn/T/ipykernel_42299/982137659.py:74: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_rebuilt'] = df['is_rebuilt'].fillna(False)


In [46]:
df.head(100)

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,description,title,rent_value,credit_value,rent_credit_transform,building_size,floor,rooms_count,has_elevator,has_warehouse,has_parking,construction_year,is_rebuilt,location_latitude,location_longitude
0,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,26000000.0,7.500000e+08,False,132.0,3,سه,True,True,True,۱۴۰۱,False,35.703865,51.373459
1,residential-rent,apartment-rent,tehran,gisha,2024-08-01 00:00:00,‼️‼️ارزنده ترین واحد منطقه ‼️‼️ \n\n✅ 91 متر ب...,گیشا / ۹۱ متر /فرعی زوج مشرف به پارک گفتگو/ ار...,0.0,1.200000e+09,True,91.0,1,دو,False,True,True,۱۳۸۵,True,35.733952,51.380608
2,residential-rent,apartment-rent,tehran,majidiyeh,2024-11-01 00:00:00,60 متر آپارتمان \n\nیک خوابه \n\nطبقه دوم \n\n...,60 متر یک خوابه,15000000.0,1.500000e+08,False,60.0,2,یک,False,False,True,۱۳۸۳,True,35.735909,51.464397
3,residential-rent,apartment-rent,tehran,west-tehranpars,2024-07-01 00:00:00,✅.....املاک بزرگ تهرانپارس:سام.....✅\n\n\n86 م...,86متر*2خواب*کلید نخورده*فول*بهترین لوکیشن,0.0,1.500000e+09,True,86.0,2,دو,True,True,True,۱۴۰۰,True,35.739414,51.525150
4,residential-rent,apartment-rent,tehran,tehran-kerman,2024-06-01 00:00:00,این واحد در طبقه همکف یک ساختمان دوطبقه واقع ش...,اپارتمان ۸۰ متری بازسازی کامل یکخواب,10000000.0,2.500000e+08,False,80.0,0,یک,False,True,False,۱۳۷۵,True,35.729465,51.468460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,residential-rent,apartment-rent,tehran,shahin,2024-06-01 00:00:00,باهرسلیقه و باهربودجه در خدمت شماعزیزان هستیم ...,۱۰۰متر جنت اباد سالن پرده خور قابل تبدیل,6000000.0,1.300000e+09,True,100.0,5,دو,True,True,True,۱۳۹۸,False,35.759972,51.311829
96,residential-rent,apartment-rent,tehran,niavaran,2024-06-01 00:00:00,✅با سلام و عرض ادب\n\n✅۳۲۰ متر آپارتمان کلید ن...,نیاوران ، ۳۲۰ متر ، نوساز ، فول مشاعات ، تراس ...,175000000.0,2.000000e+09,False,320.0,5,چهار,True,True,True,۱۴۰۳,False,35.806072,51.460337
97,residential-rent,apartment-rent,tehran,tajrish,2024-05-01 00:00:00,فول بازسازی شده مدرن اتاق ها پر از کمد استثنا...,110 متر فناخسرو دربند اجاره,100000.0,2.200000e+09,False,110.0,1,دو,True,True,True,۱۳۸۹,True,35.803616,51.427773
98,residential-rent,apartment-rent,tehran,darvazeh-shemiran,2024-09-01 00:00:00,تک واحدی\nطبقه دوم\nتخلیه و اماده تحویل\nمحدود...,50متر یک خواب,0.0,5.500000e+08,False,50.0,2,یک,True,True,False,۱۳۸۸,False,35.691914,51.444641


In [48]:
# Create `jalali_date` from `created_at_month` (handles Persian/Arabic-Indic digits), then drop `created_at_month`
import re

def persian_to_int_month(val):
    if pd.isna(val):
        return np.nan
    s = str(val)
    mapping = {
        '۰':'0','۱':'1','۲':'2','۳':'3','۴':'4','۵':'5','۶':'6','۷':'7','۸':'8','۹':'9',
        '٠':'0','١':'1','٢':'2','٣':'3','٤':'4','٥':'5','٦':'6','٧':'7','٨':'8','٩':'9'
    }
    trans = str.maketrans(mapping)
    s2 = s.translate(trans)
    s2 = re.sub(r"[^0-9]", "", s2)
    if s2 == '':
        return np.nan
    try:
        return int(s2)
    except Exception:
        return np.nan

if 'created_at_month' in df.columns:
    df['_created_month_num'] = df['created_at_month'].apply(persian_to_int_month)
    # Keep only valid months 1-12
    df.loc[~df['_created_month_num'].between(1,12), '_created_month_num'] = np.nan
    # Build Jalali date string using year 1404 (mid-month day as 01)
    df['jalali_date'] = df['_created_month_num'].apply(lambda m: f"1404-{int(m):02d}-01" if not pd.isna(m) else np.nan)
    df = df.drop(columns=['created_at_month','_created_month_num'])
    print("Created 'jalali_date' and dropped 'created_at_month'. Sample values:")
    print(df['jalali_date'].head().to_list())
else:
    print("Column 'created_at_month' not found — no changes made.")

Created 'jalali_date' and dropped 'created_at_month'. Sample values:
[nan, nan, nan, nan, nan]


In [61]:
# Parse created_at_month (Gregorian datetime) and convert to Jalali date
import re
from datetime import datetime

print("Checking created_at_month column...")
if 'created_at_month' in df.columns:
    print(f"Sample values: {df['created_at_month'].head(10).to_list()}")
    
    # Parse datetime and extract month/year
    df['_temp_datetime'] = pd.to_datetime(df['created_at_month'], errors='coerce')
    
    # Simple Gregorian to Jalali conversion (approximation for 2024)
    # For accurate conversion, we'd need jdatetime library, but let's use a simple mapping
    # 2024 Gregorian roughly maps to 1402-1403 Jalali
    # Each Gregorian month maps to Jalali months with some offset
    
    # Simplified approach: extract the month from Gregorian date
    # Since data is from 2024, we can map to 1403 Jalali year
    df['_gregorian_month'] = df['_temp_datetime'].dt.month
    df['_gregorian_year'] = df['_temp_datetime'].dt.year
    
    # Create jalali_date using the Gregorian month as Jalali month (approximate)
    # For better accuracy, you'd need jdatetime library
    # But for now, let's just use the month number
    df['jalali_date'] = df['_gregorian_month'].apply(
        lambda m: f"1403-{int(m):02d}-01" if not pd.isna(m) else np.nan
    )
    
    # Drop temporary columns
    df.drop(columns=['_temp_datetime', '_gregorian_month', '_gregorian_year'], inplace=True)
    
    parsed = df['jalali_date'].notna().sum()
    print(f"\nCreated jalali_date for {parsed} / {len(df)} rows")
    print(f"Sample jalali_date values: {df['jalali_date'].head(10).to_list()}")
    print(f"Unique jalali_date values: {sorted(df['jalali_date'].dropna().unique())}")
    
    print(f"\nFinal df shape: {df.shape}")
    print(f"Columns: {df.columns.to_list()}")
else:
    print("Column 'created_at_month' not found.")

Checking created_at_month column...
Column 'created_at_month' not found.


In [52]:
df.head(100)

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,description,title,rent_value,credit_value,rent_credit_transform,building_size,...,rooms_count,has_elevator,has_warehouse,has_parking,construction_year,is_rebuilt,location_latitude,location_longitude,property_age,jalali_date
0,residential-rent,apartment-rent,tehran,tohid,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,26000000.0,7.500000e+08,False,132.0,...,سه,True,True,True,۱۴۰۱,False,35.703865,51.373459,3.0,NaN
1,residential-rent,apartment-rent,tehran,gisha,‼️‼️ارزنده ترین واحد منطقه ‼️‼️ \n\n✅ 91 متر ب...,گیشا / ۹۱ متر /فرعی زوج مشرف به پارک گفتگو/ ار...,0.0,1.200000e+09,True,91.0,...,دو,False,True,True,۱۳۸۵,True,35.733952,51.380608,19.0,NaN
2,residential-rent,apartment-rent,tehran,majidiyeh,60 متر آپارتمان \n\nیک خوابه \n\nطبقه دوم \n\n...,60 متر یک خوابه,15000000.0,1.500000e+08,False,60.0,...,یک,False,False,True,۱۳۸۳,True,35.735909,51.464397,21.0,NaN
3,residential-rent,apartment-rent,tehran,west-tehranpars,✅.....املاک بزرگ تهرانپارس:سام.....✅\n\n\n86 م...,86متر*2خواب*کلید نخورده*فول*بهترین لوکیشن,0.0,1.500000e+09,True,86.0,...,دو,True,True,True,۱۴۰۰,True,35.739414,51.525150,4.0,NaN
4,residential-rent,apartment-rent,tehran,tehran-kerman,این واحد در طبقه همکف یک ساختمان دوطبقه واقع ش...,اپارتمان ۸۰ متری بازسازی کامل یکخواب,10000000.0,2.500000e+08,False,80.0,...,یک,False,True,False,۱۳۷۵,True,35.729465,51.468460,29.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,residential-rent,apartment-rent,tehran,shahin,باهرسلیقه و باهربودجه در خدمت شماعزیزان هستیم ...,۱۰۰متر جنت اباد سالن پرده خور قابل تبدیل,6000000.0,1.300000e+09,True,100.0,...,دو,True,True,True,۱۳۹۸,False,35.759972,51.311829,6.0,NaN
96,residential-rent,apartment-rent,tehran,niavaran,✅با سلام و عرض ادب\n\n✅۳۲۰ متر آپارتمان کلید ن...,نیاوران ، ۳۲۰ متر ، نوساز ، فول مشاعات ، تراس ...,175000000.0,2.000000e+09,False,320.0,...,چهار,True,True,True,۱۴۰۳,False,35.806072,51.460337,1.0,NaN
97,residential-rent,apartment-rent,tehran,tajrish,فول بازسازی شده مدرن اتاق ها پر از کمد استثنا...,110 متر فناخسرو دربند اجاره,100000.0,2.200000e+09,False,110.0,...,دو,True,True,True,۱۳۸۹,True,35.803616,51.427773,15.0,NaN
98,residential-rent,apartment-rent,tehran,darvazeh-shemiran,تک واحدی\nطبقه دوم\nتخلیه و اماده تحویل\nمحدود...,50متر یک خواب,0.0,5.500000e+08,False,50.0,...,یک,True,True,False,۱۳۸۸,False,35.691914,51.444641,16.0,NaN


In [56]:
# Reload the data to get created_at_month back
print("Reloading data to restore created_at_month column...")
df = pd.read_csv('/Users/yashakhorsandi/Desktop/پروژه دیوار/real_estate_ads.csv')
print(f"Loaded df shape: {df.shape}")
print(f"'created_at_month' present? {'created_at_month' in df.columns}")

Reloading data to restore created_at_month column...


/var/folders/j2/6c3r7nc116sgr13bnd1dxtth0000gn/T/ipykernel_42299/658454903.py:3: DtypeWarning: Columns (10,26,28,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/yashakhorsandi/Desktop/پروژه دیوار/real_estate_ads.csv')


Loaded df shape: (1000000, 60)
'created_at_month' present? True


In [62]:
# Check if we need to restore created_at_month and reprocess
print(f"Current columns: {df.columns.to_list()}")
print(f"\n'created_at_month' in df: {'created_at_month' in df.columns}")
print(f"'jalali_date' in df: {'jalali_date' in df.columns}")

if 'jalali_date' in df.columns:
    print(f"\njalali_date non-null count: {df['jalali_date'].notna().sum()} / {len(df)}")
    print(f"Sample jalali_date: {df['jalali_date'].head(10).to_list()}")

Current columns: ['cat2_slug', 'cat3_slug', 'city_slug', 'neighborhood_slug', 'description', 'title', 'rent_value', 'credit_value', 'rent_credit_transform', 'building_size', 'floor', 'rooms_count', 'has_elevator', 'has_warehouse', 'has_parking', 'construction_year', 'is_rebuilt', 'location_latitude', 'location_longitude', 'property_age', 'jalali_date']

'created_at_month' in df: False
'jalali_date' in df: True

jalali_date non-null count: 0 / 64198
Sample jalali_date: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [63]:
# COMPLETE PIPELINE: Reload data and apply all cleaning steps WITH jalali_date creation
print("=" * 60)
print("REBUILDING COMPLETE PIPELINE WITH JALALI_DATE")
print("=" * 60)

# 1. Load data
df = pd.read_csv('/Users/yashakhorsandi/Desktop/پروژه دیوار/real_estate_ads.csv', low_memory=False)
print(f"\n1. Loaded data: {df.shape}")

# 2. Drop unwanted columns (first batch)
cols_to_drop = [
    'location_radius', 'regular_person_capacity', 'extra_person_capacity',
    'cost_per_extra_person', 'rent_price_on_regular_days',
    'rent_price_on_special_days', 'rent_price_at_weekends', 'has_water', 
    'has_warm_water_provider', 'has_electricity', 'has_gas',
    'has_heating_system', 'has_cooling_system', 'has_restroom',
    'has_security_guard', 'has_barbecue', 'building_direction', 'has_pool',
    'has_jacuzzi', 'has_sauna', 'floor_material', 'property_type'
]
df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)
print(f"2. After first column drops: {df.shape}")

# 3. Filter for Tehran apartment rentals
df = df[
    (df['city_slug'] == 'tehran') & 
    (df['cat2_slug'] == 'residential-rent') & 
    (df['cat3_slug'] == 'apartment-rent')
].copy().reset_index(drop=True)
print(f"3. After Tehran filter: {df.shape}")

# 4. Drop transformable/deed columns
extra_drop = [
    'transformable_price','transformable_credit','transformed_credit',
    'transformable_rent','transformed_rent','land_size','deed_type','has_business_deed'
]
df.drop(columns=[c for c in extra_drop if c in df.columns], inplace=True)
print(f"4. After transformable drops: {df.shape}")

# 5. Drop rent-related columns
drop_cols_2 = ['rent_mode', 'rent_to_single', 'rent_type', 'price_mode', 'credit_mode']
df.drop(columns=[c for c in drop_cols_2 if c in df.columns], inplace=True)
print(f"5. After rent column drops: {df.shape}")

# 6. Drop price_value and rows missing rent/credit
if 'price_value' in df.columns:
    df.drop(columns=['price_value'], inplace=True)
df.dropna(subset=['rent_value', 'credit_value'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(f"6. After dropping missing rent/credit: {df.shape}")

# 7. Impute lat/lon
df['location_latitude'] = pd.to_numeric(df['location_latitude'], errors='coerce')
df['location_longitude'] = pd.to_numeric(df['location_longitude'], errors='coerce')
centroids = df.groupby('neighborhood_slug')[['location_latitude','location_longitude']].mean()
df['location_latitude'].fillna(df['neighborhood_slug'].map(centroids['location_latitude']), inplace=True)
df['location_longitude'].fillna(df['neighborhood_slug'].map(centroids['location_longitude']), inplace=True)
df['location_latitude'].fillna(df['location_latitude'].mean(), inplace=True)
df['location_longitude'].fillna(df['location_longitude'].mean(), inplace=True)
print(f"7. After lat/lon imputation: {df.shape}")

# 8. Drop additional columns
drop_cols_3 = ['user_type', 'total_floors_count', 'unit_per_floor', 'has_balcony']
df.drop(columns=[c for c in drop_cols_3 if c in df.columns], inplace=True)
print(f"8. After additional column drops: {df.shape}")

# 9. Impute is_rebuilt
if 'is_rebuilt' in df.columns:
    df['is_rebuilt'].fillna(False, inplace=True)
print(f"9. After is_rebuilt imputation: {df.shape}")

# 10. Compute property_age
def persian_to_integer(val):
    if pd.isna(val):
        return np.nan
    s = str(val)
    mapping = {
        '۰':'0','۱':'1','۲':'2','۳':'3','۴':'4','۵':'5','۶':'6','۷':'7','۸':'8','۹':'9',
        '٠':'0','١':'1','٢':'2','٣':'3','٤':'4','٥':'5','٦':'6','٧':'7','٨':'8','٩':'9'
    }
    s2 = s.translate(str.maketrans(mapping))
    s2 = re.sub(r"[^0-9]", "", s2)
    return int(s2) if s2 else np.nan

if 'construction_year' in df.columns:
    df['_construction_year_num'] = df['construction_year'].apply(persian_to_integer)
    df['property_age'] = 1404 - df['_construction_year_num']
    df.loc[df['property_age'] < 0, 'property_age'] = np.nan
    df.loc[df['property_age'] > 1404, 'property_age'] = np.nan
    df.drop(columns=['_construction_year_num'], inplace=True)
print(f"10. After property_age computation: {df.shape}")

# 11. CREATE JALALI_DATE from created_at_month
if 'created_at_month' in df.columns:
    print("\n11. Processing created_at_month to create jalali_date...")
    df['_temp_datetime'] = pd.to_datetime(df['created_at_month'], errors='coerce')
    df['_gregorian_month'] = df['_temp_datetime'].dt.month
    df['jalali_date'] = df['_gregorian_month'].apply(
        lambda m: f"1403-{int(m):02d}-01" if not pd.isna(m) else np.nan
    )
    df.drop(columns=['created_at_month', '_temp_datetime', '_gregorian_month'], inplace=True)
    jalali_count = df['jalali_date'].notna().sum()
    print(f"    Created jalali_date for {jalali_count} / {len(df)} rows")
    print(f"    Sample: {df['jalali_date'].head(5).to_list()}")
else:
    print("\n11. WARNING: created_at_month not found!")

print(f"\n{'='*60}")
print(f"FINAL CLEANED DATASET")
print(f"{'='*60}")
print(f"Shape: {df.shape}")
print(f"Columns ({len(df.columns)}): {df.columns.to_list()}")
print(f"\njalali_date stats:")
print(f"  Non-null: {df['jalali_date'].notna().sum()}")
print(f"  Unique values: {sorted(df['jalali_date'].dropna().unique())}")

REBUILDING COMPLETE PIPELINE WITH JALALI_DATE

1. Loaded data: (1000000, 60)
2. After first column drops: (1000000, 38)
3. After Tehran filter: (64630, 38)
4. After transformable drops: (64630, 30)
5. After rent column drops: (64630, 25)
6. After dropping missing rent/credit: (64198, 24)
7. After lat/lon imputation: (64198, 24)
8. After additional column drops: (64198, 20)
9. After is_rebuilt imputation: (64198, 20)


/var/folders/j2/6c3r7nc116sgr13bnd1dxtth0000gn/T/ipykernel_42299/1375754055.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['location_latitude'].fillna(df['neighborhood_slug'].map(centroids['location_latitude']), inplace=True)
/var/folders/j2/6c3r7nc116sgr13bnd1dxtth0000gn/T/ipykernel_42299/1375754055.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because 

10. After property_age computation: (64198, 21)

11. Processing created_at_month to create jalali_date...
    Created jalali_date for 64198 / 64198 rows
    Sample: ['1403-10-01', '1403-08-01', '1403-11-01', '1403-07-01', '1403-06-01']

FINAL CLEANED DATASET
Shape: (64198, 21)
Columns (21): ['cat2_slug', 'cat3_slug', 'city_slug', 'neighborhood_slug', 'description', 'title', 'rent_value', 'credit_value', 'rent_credit_transform', 'building_size', 'floor', 'rooms_count', 'has_elevator', 'has_warehouse', 'has_parking', 'construction_year', 'is_rebuilt', 'location_latitude', 'location_longitude', 'property_age', 'jalali_date']

jalali_date stats:
  Non-null: 64198
  Unique values: ['1403-01-01', '1403-02-01', '1403-03-01', '1403-04-01', '1403-05-01', '1403-06-01', '1403-07-01', '1403-08-01', '1403-09-01', '1403-10-01', '1403-11-01', '1403-12-01']


In [64]:
# Save the final cleaned dataset with jalali_date
output_path = '/Users/yashakhorsandi/Desktop/پروژه دیوار/tehran_rent_cleaned.csv'
df.to_csv(output_path, index=False)
print(f"✅ Saved cleaned dataset to: {output_path}")
print(f"   Shape: {df.shape}")
print(f"   Columns: {len(df.columns)}")

✅ Saved cleaned dataset to: /Users/yashakhorsandi/Desktop/پروژه دیوار/tehran_rent_cleaned.csv
   Shape: (64198, 21)
   Columns: 21


In [65]:
df.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,description,title,rent_value,credit_value,rent_credit_transform,building_size,...,rooms_count,has_elevator,has_warehouse,has_parking,construction_year,is_rebuilt,location_latitude,location_longitude,property_age,jalali_date
0,residential-rent,apartment-rent,tehran,tohid,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,26000000.0,7.500000e+08,False,132.0,...,سه,True,True,True,۱۴۰۱,False,35.703865,51.373459,3.0,1403-10-01
1,residential-rent,apartment-rent,tehran,gisha,‼️‼️ارزنده ترین واحد منطقه ‼️‼️ \n\n✅ 91 متر ب...,گیشا / ۹۱ متر /فرعی زوج مشرف به پارک گفتگو/ ار...,0.0,1.200000e+09,True,91.0,...,دو,False,True,True,۱۳۸۵,True,35.733952,51.380608,19.0,1403-08-01
2,residential-rent,apartment-rent,tehran,majidiyeh,60 متر آپارتمان \n\nیک خوابه \n\nطبقه دوم \n\n...,60 متر یک خوابه,15000000.0,1.500000e+08,False,60.0,...,یک,False,False,True,۱۳۸۳,True,35.735909,51.464397,21.0,1403-11-01
3,residential-rent,apartment-rent,tehran,west-tehranpars,✅.....املاک بزرگ تهرانپارس:سام.....✅\n\n\n86 م...,86متر*2خواب*کلید نخورده*فول*بهترین لوکیشن,0.0,1.500000e+09,True,86.0,...,دو,True,True,True,۱۴۰۰,True,35.739414,51.525150,4.0,1403-07-01
4,residential-rent,apartment-rent,tehran,tehran-kerman,این واحد در طبقه همکف یک ساختمان دوطبقه واقع ش...,اپارتمان ۸۰ متری بازسازی کامل یکخواب,10000000.0,2.500000e+08,False,80.0,...,یک,False,True,False,۱۳۷۵,True,35.729465,51.468460,29.0,1403-06-01


In [66]:
# Drop 'is_rebuilt' column and update df
if 'is_rebuilt' in df.columns:
    df = df.drop(columns=['is_rebuilt'])
    print("Dropped column: 'is_rebuilt'")
else:
    print("Column 'is_rebuilt' not found.")

print(f"Updated df shape: {df.shape}")
print(f"Current columns: {df.columns.to_list()}")

Dropped column: 'is_rebuilt'
Updated df shape: (64198, 20)
Current columns: ['cat2_slug', 'cat3_slug', 'city_slug', 'neighborhood_slug', 'description', 'title', 'rent_value', 'credit_value', 'rent_credit_transform', 'building_size', 'floor', 'rooms_count', 'has_elevator', 'has_warehouse', 'has_parking', 'construction_year', 'location_latitude', 'location_longitude', 'property_age', 'jalali_date']


In [67]:
# Extract NLP features from Farsi title and description columns
import re

print("Extracting features from Persian title and description...")

# Combine title and description for comprehensive text analysis
df['combined_text'] = df['title'].fillna('') + ' ' + df['description'].fillna('')

# Define Persian keywords to search for (common amenities and property features)
keywords = {
    'has_parking_text': r'پارکینگ',
    'has_elevator_text': r'آسانسور',
    'has_storage_text': r'انباری',
    'has_balcony_text': r'بالکن|تراس',
    'is_furnished': r'مبله|مبل',
    'is_renovated': r'بازسازی|نوساز|نو\s*ساز',
    'has_view': r'ویو|منظره',
    'has_garden': r'باغ|حیاط',
    'is_duplex': r'دوبلکس|دوبلکس',
    'north_facing': r'شمالی',
    'south_facing': r'جنوبی',
    'has_master_room': r'مستر|مسترروم',
    'luxurious': r'لوکس|لاکچری|مجلل',
    'quiet_area': r'ساکت|آرام',
    'near_metro': r'مترو|متروپلیتن',
    'near_park': r'پارک',
    'near_shopping': r'مرکز خرید|پاساژ',
    'vip_building': r'وی\s*آی\s*پی|VIP',
    'modern': r'مدرن|جدید',
    'traditional': r'سنتی|قدیمی'
}

# Extract binary features using vectorized string operations
for feature_name, pattern in keywords.items():
    df[feature_name] = df['combined_text'].str.contains(pattern, case=False, na=False, regex=True).astype(int)
    count = df[feature_name].sum()
    print(f"  {feature_name}: {count} properties ({count/len(df)*100:.1f}%)")

# Extract numeric features from text
print("\nExtracting numeric features from text...")

# Floor mentions in text (backup if floor column has missing values)
def extract_floor_from_text(text):
    if pd.isna(text):
        return np.nan
    # Look for patterns like "طبقه 3" or "طبقه سوم"
    match = re.search(r'طبقه\s*(\d+)', str(text))
    if match:
        return int(match.group(1))
    return np.nan

df['floor_from_text'] = df['combined_text'].apply(extract_floor_from_text)
extracted_floors = df['floor_from_text'].notna().sum()
print(f"  floor_from_text: extracted from {extracted_floors} descriptions")

# Count of amenity mentions (richness of description)
df['amenity_count'] = sum(df[col] for col in keywords.keys())
print(f"  amenity_count: mean={df['amenity_count'].mean():.2f}, max={df['amenity_count'].max()}")

# Text length features (proxy for detail level)
df['title_length'] = df['title'].fillna('').str.len()
df['description_length'] = df['description'].fillna('').str.len()
print(f"  title_length: mean={df['title_length'].mean():.0f} chars")
print(f"  description_length: mean={df['description_length'].mean():.0f} chars")

# Word count
df['description_words'] = df['description'].fillna('').str.split().str.len()
print(f"  description_words: mean={df['description_words'].mean():.0f} words")

print(f"\n✅ NLP feature extraction complete!")
print(f"Added {len(keywords) + 5} new features from text")
print(f"Current df shape: {df.shape}")

Extracting features from Persian title and description...
  has_parking_text: 28147 properties (43.8%)
  has_elevator_text: 15353 properties (23.9%)
  has_storage_text: 22021 properties (34.3%)
  has_balcony_text: 21194 properties (33.0%)
  is_furnished: 3544 properties (5.5%)
  is_renovated: 15189 properties (23.7%)
  has_view: 6498 properties (10.1%)
  has_garden: 5555 properties (8.7%)
  is_duplex: 225 properties (0.4%)
  north_facing: 5262 properties (8.2%)
  south_facing: 4354 properties (6.8%)
  has_master_room: 8683 properties (13.5%)
  luxurious: 5350 properties (8.3%)
  quiet_area: 6261 properties (9.8%)
  near_metro: 9281 properties (14.5%)
  near_park: 31821 properties (49.6%)
  near_shopping: 801 properties (1.2%)
  vip_building: 309 properties (0.5%)
  modern: 2549 properties (4.0%)
  traditional: 488 properties (0.8%)

Extracting numeric features from text...
  floor_from_text: extracted from 6732 descriptions
  amenity_count: mean=3.00, max=14
  title_length: mean=32 cha

In [68]:
# Create total_monthly_rent column: rent_value + (credit_value * 0.03)
print("Creating total_monthly_rent column...")

# Calculate credit equivalent monthly rent (3% of credit_value)
df['total_monthly_rent'] = df['rent_value'] + (df['credit_value'] * 0.03)

print(f"✅ Created 'total_monthly_rent' column")
print(f"\nStatistics:")
print(f"  Min: {df['total_monthly_rent'].min():,.0f}")
print(f"  Mean: {df['total_monthly_rent'].mean():,.0f}")
print(f"  Median: {df['total_monthly_rent'].median():,.0f}")
print(f"  Max: {df['total_monthly_rent'].max():,.0f}")

print(f"\nSample values:")
sample = df[['rent_value', 'credit_value', 'total_monthly_rent']].head(10)
print(sample.to_string())

print(f"\nCurrent df shape: {df.shape}")

Creating total_monthly_rent column...
✅ Created 'total_monthly_rent' column

Statistics:
  Min: 1,030
  Mean: 4,263,869,311
  Median: 33,000,000
  Max: 114,444,444,444,444

Sample values:
   rent_value  credit_value  total_monthly_rent
0  26000000.0  7.500000e+08          48500000.0
1         0.0  1.200000e+09          36000000.0
2  15000000.0  1.500000e+08          19500000.0
3         0.0  1.500000e+09          45000000.0
4  10000000.0  2.500000e+08          17500000.0
5  15000000.0  3.000000e+08          24000000.0
6         0.0  2.500000e+09          75000000.0
7   4000000.0  4.500000e+08          17500000.0
8  60000000.0  1.500000e+09         105000000.0
9         0.0  1.200000e+09          36000000.0

Current df shape: (64198, 47)


In [69]:
df.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,description,title,rent_value,credit_value,rent_credit_transform,building_size,...,near_shopping,vip_building,modern,traditional,floor_from_text,amenity_count,title_length,description_length,description_words,total_monthly_rent
0,residential-rent,apartment-rent,tehran,tohid,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,26000000.0,7.500000e+08,False,132.0,...,0,0,0,0,NaN,0,24,15,3,48500000.0
1,residential-rent,apartment-rent,tehran,gisha,‼️‼️ارزنده ترین واحد منطقه ‼️‼️ \n\n✅ 91 متر ب...,گیشا / ۹۱ متر /فرعی زوج مشرف به پارک گفتگو/ ار...,0.0,1.200000e+09,True,91.0,...,0,0,0,0,NaN,2,50,383,72,36000000.0
2,residential-rent,apartment-rent,tehran,majidiyeh,60 متر آپارتمان \n\nیک خوابه \n\nطبقه دوم \n\n...,60 متر یک خوابه,15000000.0,1.500000e+08,False,60.0,...,0,0,0,0,NaN,4,15,141,27,19500000.0
3,residential-rent,apartment-rent,tehran,west-tehranpars,✅.....املاک بزرگ تهرانپارس:سام.....✅\n\n\n86 م...,86متر*2خواب*کلید نخورده*فول*بهترین لوکیشن,0.0,1.500000e+09,True,86.0,...,0,0,0,0,NaN,8,41,861,135,45000000.0
4,residential-rent,apartment-rent,tehran,tehran-kerman,این واحد در طبقه همکف یک ساختمان دوطبقه واقع ش...,اپارتمان ۸۰ متری بازسازی کامل یکخواب,10000000.0,2.500000e+08,False,80.0,...,0,0,0,0,NaN,6,36,195,41,17500000.0


In [70]:
# Prepare data for modeling: total_monthly_rent as target
print("="*60)
print("PREPARING DATA FOR MODELING")
print("="*60)

# Check target variable
print(f"\nTarget variable: total_monthly_rent")
print(f"  Non-null count: {df['total_monthly_rent'].notna().sum()} / {len(df)}")
print(f"  Min: {df['total_monthly_rent'].min():,.0f}")
print(f"  Median: {df['total_monthly_rent'].median():,.0f}")
print(f"  Mean: {df['total_monthly_rent'].mean():,.0f}")
print(f"  Max: {df['total_monthly_rent'].max():,.0f}")

# Remove extreme outliers (likely data errors)
q1 = df['total_monthly_rent'].quantile(0.01)
q99 = df['total_monthly_rent'].quantile(0.99)
before_outliers = len(df)
df_model = df[(df['total_monthly_rent'] >= q1) & (df['total_monthly_rent'] <= q99)].copy()
print(f"\nRemoved outliers outside 1st-99th percentile range:")
print(f"  Before: {before_outliers} rows")
print(f"  After: {len(df_model)} rows")
print(f"  Removed: {before_outliers - len(df_model)} rows")
print(f"  Valid range: {q1:,.0f} to {q99:,.0f}")

# Select features for modeling
# Drop non-numeric and ID columns
cols_to_drop_for_model = [
    'cat2_slug', 'cat3_slug', 'city_slug', 'neighborhood_slug',
    'title', 'description', 'combined_text',
    'construction_year',  # we have property_age instead
    'jalali_date',  # date string, not numeric
    'rent_value', 'credit_value',  # components of target, avoid leakage
    'rent_credit_transform'  # likely related to target
]

# Create feature matrix
X = df_model.drop(columns=cols_to_drop_for_model + ['total_monthly_rent'], errors='ignore')
y = df_model['total_monthly_rent']

print(f"\nFeature matrix:")
print(f"  Shape: {X.shape}")
print(f"  Features ({X.shape[1]}): {X.columns.to_list()}")

# Check for any remaining non-numeric columns
non_numeric = X.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric:
    print(f"\n⚠️  Warning: Found non-numeric columns: {non_numeric}")
    print("  Dropping these columns...")
    X = X.select_dtypes(include=[np.number])
    print(f"  Updated shape: {X.shape}")

# Handle missing values in features
missing_counts = X.isnull().sum()
cols_with_missing = missing_counts[missing_counts > 0]
if len(cols_with_missing) > 0:
    print(f"\nColumns with missing values:")
    for col, count in cols_with_missing.items():
        pct = count / len(X) * 100
        print(f"  {col}: {count} ({pct:.1f}%)")
    
    # Simple imputation: fill with median for numeric
    print("\nImputing missing values with median...")
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='median')
    X_imputed = pd.DataFrame(
        imputer.fit_transform(X),
        columns=X.columns,
        index=X.index
    )
    X = X_imputed
    print(f"  Missing values after imputation: {X.isnull().sum().sum()}")
else:
    print(f"\n✅ No missing values in features")

print(f"\nFinal modeling dataset:")
print(f"  Features (X): {X.shape}")
print(f"  Target (y): {y.shape}")
print(f"  Target range: {y.min():,.0f} to {y.max():,.0f}")

PREPARING DATA FOR MODELING

Target variable: total_monthly_rent
  Non-null count: 64198 / 64198
  Min: 1,030
  Median: 33,000,000
  Mean: 4,263,869,311
  Max: 114,444,444,444,444

Removed outliers outside 1st-99th percentile range:
  Before: 64198 rows
  After: 62944 rows
  Removed: 1254 rows
  Valid range: 1,300,000 to 360,000,000

Feature matrix:
  Shape: (62944, 34)
  Features (34): ['building_size', 'floor', 'rooms_count', 'has_elevator', 'has_warehouse', 'has_parking', 'location_latitude', 'location_longitude', 'property_age', 'has_parking_text', 'has_elevator_text', 'has_storage_text', 'has_balcony_text', 'is_furnished', 'is_renovated', 'has_view', 'has_garden', 'is_duplex', 'north_facing', 'south_facing', 'has_master_room', 'luxurious', 'quiet_area', 'near_metro', 'near_park', 'near_shopping', 'vip_building', 'modern', 'traditional', 'floor_from_text', 'amenity_count', 'title_length', 'description_length', 'description_words']

⚠️  Warning: Found non-numeric columns: ['floor', 

In [71]:
# Train-Test-Validation Split
from sklearn.model_selection import train_test_split

print("="*60)
print("TRAIN-TEST-VALIDATION SPLIT")
print("="*60)

# Split: 70% train, 15% validation, 15% test
# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42
)

# Second split: split temp into 50-50 (15% validation, 15% test of total)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)

print(f"\nDataset splits:")
print(f"  Training set:   {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Validation set: {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test set:       {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"  Total:          {len(X):,} samples")

print(f"\nTarget statistics by split:")
for name, y_split in [('Train', y_train), ('Validation', y_val), ('Test', y_test)]:
    print(f"\n  {name}:")
    print(f"    Mean:   {y_split.mean():,.0f}")
    print(f"    Median: {y_split.median():,.0f}")
    print(f"    Std:    {y_split.std():,.0f}")

print(f"\n✅ Data ready for model training!")

TRAIN-TEST-VALIDATION SPLIT

Dataset splits:
  Training set:   44,060 samples (70.0%)
  Validation set: 9,442 samples (15.0%)
  Test set:       9,442 samples (15.0%)
  Total:          62,944 samples

Target statistics by split:

  Train:
    Mean:   49,133,376
    Median: 33,000,000
    Std:    50,478,890

  Validation:
    Mean:   49,485,456
    Median: 33,100,000
    Std:    50,298,855

  Test:
    Mean:   49,429,938
    Median: 33,000,000
    Std:    50,825,417

✅ Data ready for model training!


In [72]:
# Train baseline models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time

print("="*60)
print("TRAINING BASELINE MODELS")
print("="*60)

models = {
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Training {name}...")
    print('='*60)
    
    # Train
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Predict on all sets
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    results[name] = {
        'train_mae': mean_absolute_error(y_train, y_train_pred),
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'train_r2': r2_score(y_train, y_train_pred),
        'val_mae': mean_absolute_error(y_val, y_val_pred),
        'val_rmse': np.sqrt(mean_squared_error(y_val, y_val_pred)),
        'val_r2': r2_score(y_val, y_val_pred),
        'test_mae': mean_absolute_error(y_test, y_test_pred),
        'test_rmse': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'test_r2': r2_score(y_test, y_test_pred),
        'train_time': train_time
    }
    
    print(f"\nTraining time: {train_time:.2f}s")
    print(f"\nTraining Set:")
    print(f"  MAE:  {results[name]['train_mae']:,.0f}")
    print(f"  RMSE: {results[name]['train_rmse']:,.0f}")
    print(f"  R²:   {results[name]['train_r2']:.4f}")
    print(f"\nValidation Set:")
    print(f"  MAE:  {results[name]['val_mae']:,.0f}")
    print(f"  RMSE: {results[name]['val_rmse']:,.0f}")
    print(f"  R²:   {results[name]['val_r2']:.4f}")
    print(f"\nTest Set:")
    print(f"  MAE:  {results[name]['test_mae']:,.0f}")
    print(f"  RMSE: {results[name]['test_rmse']:,.0f}")
    print(f"  R²:   {results[name]['test_r2']:.4f}")

print(f"\n{'='*60}")
print("MODEL COMPARISON SUMMARY")
print('='*60)

# Create comparison DataFrame
comparison_df = pd.DataFrame(results).T
comparison_df = comparison_df[['val_mae', 'val_rmse', 'val_r2', 'test_mae', 'test_rmse', 'test_r2', 'train_time']]
comparison_df.columns = ['Val MAE', 'Val RMSE', 'Val R²', 'Test MAE', 'Test RMSE', 'Test R²', 'Time (s)']

print("\n", comparison_df.to_string())

best_model_name = comparison_df['Val MAE'].idxmin()
print(f"\n🏆 Best model by validation MAE: {best_model_name}")
print(f"   Validation MAE: {comparison_df.loc[best_model_name, 'Val MAE']:,.0f}")
print(f"   Test MAE: {comparison_df.loc[best_model_name, 'Test MAE']:,.0f}")
print(f"   Test R²: {comparison_df.loc[best_model_name, 'Test R²']:.4f}")

TRAINING BASELINE MODELS

Training Ridge Regression...

Training time: 0.04s

Training Set:
  MAE:  23,461,574
  RMSE: 37,520,751
  R²:   0.4475

Validation Set:
  MAE:  23,389,305
  RMSE: 36,927,407
  R²:   0.4610

Test Set:
  MAE:  23,359,006
  RMSE: 37,260,866
  R²:   0.4625

Training Random Forest...

Training time: 3.85s

Training Set:
  MAE:  5,006,115
  RMSE: 8,844,360
  R²:   0.9693

Validation Set:
  MAE:  8,566,919
  RMSE: 17,483,659
  R²:   0.8792

Test Set:
  MAE:  8,650,790
  RMSE: 18,158,566
  R²:   0.8723

Training Gradient Boosting...

Training time: 9.83s

Training Set:
  MAE:  8,074,824
  RMSE: 15,675,563
  R²:   0.9036

Validation Set:
  MAE:  8,643,165
  RMSE: 17,389,324
  R²:   0.8805

Test Set:
  MAE:  8,675,693
  RMSE: 17,660,360
  R²:   0.8793

MODEL COMPARISON SUMMARY

                         Val MAE      Val RMSE    Val R²      Test MAE     Test RMSE   Test R²  Time (s)
Ridge Regression   2.338931e+07  3.692741e+07  0.460952  2.335901e+07  3.726087e+07  0.462

In [73]:
# Save the best model (Random Forest) and necessary data for the web app
import pickle
import joblib

print("Saving model and data for web app...")

# Save the trained Random Forest model
model_to_save = models['Random Forest']
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(model_to_save, f)
print("✅ Saved model to: rf_model.pkl")

# Save feature names
feature_names = X.columns.tolist()
with open('feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)
print(f"✅ Saved {len(feature_names)} feature names")

# Save neighborhood statistics (lat/lon averages)
neighborhood_stats = df_model.groupby('neighborhood_slug')[['location_latitude', 'location_longitude']].mean()
neighborhood_stats.to_csv('neighborhood_stats.csv')
print(f"✅ Saved neighborhood stats for {len(neighborhood_stats)} neighborhoods")

# Save a sample of the cleaned data for reference
df_model[['neighborhood_slug', 'building_size', 'property_age', 'total_monthly_rent']].head(100).to_csv('sample_data.csv', index=False)
print("✅ Saved sample data")

# Save model metadata
metadata = {
    'features': feature_names,
    'model_type': 'RandomForestRegressor',
    'train_samples': len(X_train),
    'test_r2': results['Random Forest']['test_r2'],
    'test_mae': results['Random Forest']['test_mae'],
    'valid_range': {
        'min': float(y.min()),
        'max': float(y.max()),
        'median': float(y.median())
    }
}

with open('model_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
print("✅ Saved model metadata")

print(f"\n{'='*60}")
print("MODEL DEPLOYMENT PACKAGE READY!")
print('='*60)
print(f"\nFiles created:")
print(f"  - rf_model.pkl (trained Random Forest model)")
print(f"  - feature_names.pkl (list of {len(feature_names)} features)")
print(f"  - neighborhood_stats.csv ({len(neighborhood_stats)} neighborhoods)")
print(f"  - model_metadata.pkl (model info and performance)")
print(f"  - sample_data.csv (100 sample rows)")
print(f"\nYou can now run the Streamlit app with: streamlit run app.py")

Saving model and data for web app...
✅ Saved model to: rf_model.pkl
✅ Saved 29 feature names
✅ Saved neighborhood stats for 342 neighborhoods
✅ Saved sample data
✅ Saved model metadata

MODEL DEPLOYMENT PACKAGE READY!

Files created:
  - rf_model.pkl (trained Random Forest model)
  - feature_names.pkl (list of 29 features)
  - neighborhood_stats.csv (342 neighborhoods)
  - model_metadata.pkl (model info and performance)
  - sample_data.csv (100 sample rows)

You can now run the Streamlit app with: streamlit run app.py
